In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType
from pyspark.sql.functions import expr, array, col, explode, arrays_zip, lit, avg, when, broadcast

# warehouse_location points to the default location for managed databases and tables
warehouse = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
jobs_df = spark.table("database.jobs")
popu_df = spark.table("database.population")
jobs_df = jobs_df.filter(col("indicador_name").like("%enrollment, secondary%"))
jobs_df = jobs_df.drop(col("pais_cod"), col("indicador_code"))
popu_df = popu_df.filter(col("cod_indicador").like("%SP.POP.TOTL%"))
popu_df = popu_df.drop(col("cod_pais"), col("cod_indicador"), col("nome_indicador"), col("Country_name"), col("Country_code"), col("Indicator_name"), col("Indicator_code"))
popu_df = popu_df.withColumnRenamed("valor", "valorI")
popu_df = popu_df.withColumnRenamed("nome_pais", "pais_name")
popu_df = popu_df.withColumn("ano", col("ano").cast(IntegerType()))

In [3]:
paises = ["%Liechtenstein%", "%Germany%", "%Denmark%", "%Switzerland%", "%Poland%", "%Czech%", 
          "%Slovakia%", "%Slovenia%", "%Hungary%", "%Austria%"]
anos = ["2010", "2011", "2012", "2013", "2014", "2015"]

jobs_df = jobs_df.filter(
    col("pais_name").like(paises[0]) |
    col("pais_name").like(paises[1]) |
    col("pais_name").like(paises[2]) |
    col("pais_name").like(paises[3]) |
    col("pais_name").like(paises[4]) |
    col("pais_name").like(paises[5]) |
    col("pais_name").like(paises[6]) |
    col("pais_name").like(paises[7]) |
    col("pais_name").like(paises[8]) |
    col("pais_name").like(paises[9]) 
).filter(
    col("ano").isin(anos) &
    col("valor").isNotNull()
)

popu_df = popu_df.filter(
    col("pais_name").like(paises[0]) |
    col("pais_name").like(paises[1]) |
    col("pais_name").like(paises[2]) |
    col("pais_name").like(paises[3]) |
    col("pais_name").like(paises[4]) |
    col("pais_name").like(paises[5]) |
    col("pais_name").like(paises[6]) |
    col("pais_name").like(paises[7]) |
    col("pais_name").like(paises[8]) |
    col("pais_name").like(paises[9]) 
).filter(
    col("ano").isin(anos) &
    col("valorI").isNotNull()
)

In [4]:
literacia_df = popu_df.join(jobs_df, ["pais_name", "ano"], "left")
literacia_df = literacia_df.withColumn("valorI", ((col("valor") / 100) * (col("valorI") / 2)).cast("int"))
literacia_df = literacia_df.withColumnRenamed("pais_name", "paises")

In [5]:
literacia_df = literacia_df.withColumn("genero",
    when(col("indicador_name").like("% female%"), "F")
    .when(col("indicador_name").like("% male%"), "M")
)

literacia_df = literacia_df.filter(col("genero").isNotNull())

literacia_df = literacia_df.drop(col("indicador_name"))

In [6]:
literacia_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/Projeto/gold/TabelaMatricula/")

In [7]:
spark.sql(
    """
    SELECT * FROM database.MatriculaGold
    """
).show()

+-------+----+---------+-------+------+
| paises| ano|    valor| valorI|genero|
+-------+----+---------+-------+------+
|Austria|2010|  96.8048|4048088|     F|
|Austria|2011| 96.06344|4030650|     F|
|Austria|2012| 95.86314|4040626|     F|
|Austria|2013|   97.274|4124331|     F|
|Austria|2014| 97.51215|4166867|     F|
|Austria|2015| 98.29222|4247550|     F|
|Austria|2010|100.76969|4213888|     M|
|Austria|2011| 99.96218|4194234|     M|
|Austria|2012| 99.58544|4197521|     M|
|Austria|2013|100.69224|4269261|     M|
|Austria|2014|101.24851|4326529|     M|
|Austria|2015|101.87345|4402307|     M|
|Denmark|2010|120.04154|3329762|     F|
|Denmark|2011|123.73984|3446508|     F|
|Denmark|2012|124.25473|3473896|     F|
|Denmark|2013|131.11752|3681079|     F|
|Denmark|2014|132.12631|3728257|     F|
|Denmark|2015|133.36955|3790017|     F|
|Denmark|2010|118.97715|3300237|     M|
|Denmark|2011|121.71088|3389996|     M|
+-------+----+---------+-------+------+
only showing top 20 rows



In [8]:
spark.stop()